Part 3 - Running the Simulation
===============================

We have completed all the necessary steps to run the Monte Carlo simulations:
- Created the `DualEnergyCTGroup` (the variable `dectgroup`).
- Imported the original CT scans for low and high energies.
- Created a segmentation voxel image (optional, to use calibration materials as a quality control; could also be imported from elsewhere).
- Created a mask voxel image (optional, to tell RockVerse which voxels to ignore in the Monte Carlo inversion; could also be imported from elsewhere).
- Populated necessary information for the calibration materials.
- Pre-calculated the set of inversion coefficients.

We can now call the `run` method to perform the full inversion. 

Remember: we are performing Monte Carlo in the Digital Rock universe! This process is computationally intensive and is designed to be run in a high-performance computing environment, such as a GPU-enabled machine or a handful of nodes in a CPU cluster. This ensures efficiency and accuracy in processing large datasets.

## Running in Command Line Mode

To run in command line mode, save a file, say `run.py`, with the following content:

```python
import rockverse as rv
dectgroup = rv.open(r'/path/to/dual_energy_ct/C04B21')
dectgroup.run()
```

and call Python with your MPI executable to run the file, e.g.:

```bash
$ mpirun -N 256 python run.py
```

or submit it as a batch script to your job scheduler, such as Slurm:

```bash
#!/bin/bash
#SBATCH -J awesome_job_name
#SBATCH --nodes=2
#SBATCH --partition <your_partition>
#SBATCH --gres=gpu:16
#SBATCH --ntasks-per-node=256
#SBATCH --output=outputlog
#SBATCH --chdir=/path/to/workdir

conda activate you_environment_with_rockverse
mpirun python3 ./run.py 
```

## Running in Jupyter with Ipyparallel

Here we'll use again a machine with 8 GPUs, and therefore we can use Ipyparallel to create a cluster
with 8 MPI processes, that will distribute the workload among the 8 GPUS. Let's create the cluster:

In [1]:
import ipyparallel as ipp

# Create an MPI cluster with 8 engines
cluster = ipp.Cluster(engines="mpi", n=8)

# Start and connect to the cluster
rc = cluster.start_and_connect_sync()

# Enable IPython magics for parallel processing
rc[:].activate()


Starting 8 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

Now we use the ``%%px`` magic to send the work to the parallel kernel:

In [ ]:
%%px --block

import rockverse as rv

# Open the group
dectgroup = rv.open('/estgf_dados/P_D/GOB7/testemunhos/C04B21_DGX')

# Call the run method to perform the calculations
dectgroup.run()

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] [2025-02-21 20:43:24] Hashing Low attenuation: 100% 16/16 [00:00<00:00, 62.83chunk/s]
[2025-02-21 20:43:24] Hashing High attenuation: 100% 16/16 [00:00<00:00, 65.49chunk/s]
[2025-02-21 20:43:25] Hashing mask: 100% 16/16 [00:00<00:00, 209.58chunk/s]
[2025-02-21 20:43:25] Hashing segmentation: 100% 16/16 [00:00<00:00, 210.28chunk/s]
[2025-02-21 20:43:25] Calibration matrices up to date.
[2025-02-21 20:43:25] Creating output images: 100% 11/11 [00:02<00:00,  4.98it/s]
[2025-02-21 20:43:30] Counting voxels: 100% 16/16 [00:03<00:00,  4.06chunk/s]
[2025-02-21 20:43:33] rho/Z inversion (chunk 16/16):  99% 38939269/39445817 [36:30:54<2:21:52, 59.50voxel/s]

After completion, you will have access to the Monte Carlo results through the
following new voxel images as attributes of dectgroup:

- ``rho_min``: Voxel image with the minimum electron density per voxel.
- ``rho_p25``: Voxel image with the the first quartile for the electron density per voxel.
- ``rho_p50``: Voxel image with the the median values for the electron density per voxel.
- ``rho_p75``: Voxel image with the the third quartile for the electron density per voxel.
- ``rho_max``: Voxel image with the maximum electron density per voxel.
- ``Z_min``: Voxel image with the minimum effective atomic number per voxel.
- ``Z_p25``: Voxel image with the the first quartile for the effective atomic number per voxel.
- ``Z_p50``: Voxel image with the the median values for the effective atomic number per voxel.
- ``Z_p75``: Voxel image with the the third quartile for the effective atomic number per voxel.
- ``Z_max``: Voxel image with the maximum effective atomic number per voxel.
- ``valid``: Voxel image with the number of valid Monte Carlo results for each voxel.